In [1]:
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
from torchvision import transforms,datasets
import torch.nn.functional as F
import numpy as np
from torch.utils.data import DataLoader, random_split


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
train_folder = './inaturalist_12K/train'
test_folder = './inaturalist_12K/val'

In [4]:
transform = transforms.Compose([
    transforms.Resize((400,400)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [5]:
train_dataset = datasets.ImageFolder(train_folder,transform=transform)
test_dataset = datasets.ImageFolder(test_folder,transform=transform)

In [6]:
class Classifier(nn.Module):
    def __init__(self,classes=10,cn1_filters=32,cn1_kernel_size=3,
                cn2_filters=64,cn2_kernel_size=3,
                cn3_filters=128,cn3_kernel_size=3,
                cn4_filters=256,cn4_kernel_size=3,
                cn5_filters=512,cn5_kernel_size=3,
                n_dense_input_neuron = 1000,
                n_dense_output_neuron = 2046
                ):
        super().__init__()
        self.cn1 = nn.Conv2d(in_channels=3,out_channels=32,kernel_size=3,padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2,stride=2)

        self.cn2 = nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=2,stride=2)

        self.cn3 = nn.Conv2d(in_channels=64,out_channels=128,kernel_size=3,padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.relu3 = nn.ReLU()
        self.maxpool3 = nn.MaxPool2d(kernel_size=2,stride=2)

        self.cn4 = nn.Conv2d(in_channels=128,out_channels=256,kernel_size=3,padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.relu4 = nn.ReLU()
        self.maxpool4 = nn.MaxPool2d(kernel_size=2,stride=2)

        self.cn5 = nn.Conv2d(in_channels=256,out_channels=512,kernel_size=3,padding=1)
        self.bn5 = nn.BatchNorm2d(512)
        self.relu5 = nn.ReLU()
        self.maxpool5 = nn.MaxPool2d(kernel_size=2,stride=2)

        self.flatten = nn.Flatten()

        self.dense_layer = nn.Linear(73728,n_dense_output_neuron)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.output_layer = nn.Linear(n_dense_output_neuron,10)

    def forward(self,data):
        out = self.cn1(data)
        out = self.bn1(out)
        out = self.relu1(out)
        out = self.maxpool1(out)

        out = self.cn2(out)
        out = self.bn2(out)
        out = self.relu2(out)
        out = self.maxpool2(out)

        out = self.cn3(out)
        out = self.bn3(out)
        out = self.relu3(out)
        out = self.maxpool3(out)

        out = self.cn4(out)
        out = self.bn4(out)
        out = self.relu4(out)
        out = self.maxpool4(out)

        out = self.cn5(out)
        out = self.bn5(out)
        out = self.relu5(out)
        out = self.maxpool5(out)

        out = self.flatten(out)

        out = self.dense_layer(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.output_layer(out)
        # return F.log_softmax(out,dim=1)
        return out

In [7]:
cnn_model = Classifier(10)
loss_func = nn.CrossEntropyLoss()
cnn_model.to(device)
optimizer = torch.optim.Adam(cnn_model.parameters(),lr=1e-5)
epochs = 1000

In [8]:
train_size = int(0.8*len(train_dataset))
val_size = len(train_dataset) - train_size

In [9]:
train_data,val_data = random_split(train_dataset,[train_size,val_size])

train_data = DataLoader(train_data,batch_size=32,shuffle=True)
val_data = DataLoader(val_data,batch_size=1,shuffle=True)

In [ ]:
for ep in range(epochs):
    cnn_model.train() 
    for i, (img, label) in enumerate(train_data):
        img = img.to(device)
        label = label.to(device)

        output = cnn_model(img)
        
        loss = loss_func(output, label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    cnn_model.eval()
    with torch.no_grad():
        total_loss_v = 0
        acc = 0
        for data, label in val_data:
            data = data.to(device)
            label = label.to(device)

            out = cnn_model(data)
            total_loss_v += loss_func(out, label).item()

            pred = out.argmax(dim=1, keepdim=True)
            acc += pred.eq(label.view_as(pred)).sum().item()

        accuracy = 100. * acc / len(val_data.dataset)

        print(f'Epoch [{ep+1}/{epochs}], Validation Loss: {total_loss_v:.4f}, Validation Accuracy: {accuracy:.2f}%')

    cnn_model.train()


Epoch [1/1000], Validation Loss: 4105.6115, Validation Accuracy: 27.45%
Epoch [2/1000], Validation Loss: 3801.5592, Validation Accuracy: 33.20%
Epoch [3/1000], Validation Loss: 3699.6916, Validation Accuracy: 35.05%
Epoch [4/1000], Validation Loss: 3699.7987, Validation Accuracy: 34.90%
Epoch [5/1000], Validation Loss: 3603.5081, Validation Accuracy: 36.90%
Epoch [6/1000], Validation Loss: 3652.6010, Validation Accuracy: 37.10%
Epoch [7/1000], Validation Loss: 3550.8026, Validation Accuracy: 38.60%
Epoch [8/1000], Validation Loss: 3643.3213, Validation Accuracy: 37.00%
Epoch [9/1000], Validation Loss: 3573.3419, Validation Accuracy: 38.40%
Epoch [10/1000], Validation Loss: 3553.9891, Validation Accuracy: 37.25%
Epoch [11/1000], Validation Loss: 3473.2477, Validation Accuracy: 40.85%
Epoch [12/1000], Validation Loss: 3516.6598, Validation Accuracy: 40.10%
Epoch [13/1000], Validation Loss: 3548.7972, Validation Accuracy: 39.30%
Epoch [14/1000], Validation Loss: 3599.8136, Validation Accu